<center> 
     <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300"> 
</center>

# Assignment: Notebook for Peer Assignment
Estimated time needed: 45 minutes


# Assignment Scenario

Congratulations! You have just been hired by a US Venture Capital firm as a data analyst.

The company is considering foreign grain markets to help meet its supply chain requirements for its recent investments in the microbrewery and microdistillery industry, which is involved with the production and distribution of craft beers and spirits.

Your first task is to provide a high level analysis of crop production in Canada. Your stakeholders want to understand the current and historical performance of certain crop types in terms of supply and price volatility. For now they are mainly interested in a macro-view of Canada's crop farming industry, and how it relates to the relative value of the Canadian and US dollars.


# Introduction

Using this R notebook you will:

1.  Understand four datasets 
2.  Load the datasets into four separate tables in a Db2 database
3.  Execute SQL queries unsing the RODBC R package to answer assignment questions 

You have already encountered two of these datasets in the previous practice lab. You will be able to reuse much of the work you did there to prepare your database tables for executing SQL queries.


# Understand the datasets

To complete the assignment problems in this notebook you will be using subsetted snapshots of two datasets from Statistics Canada, and one from the Bank of Canada. The links to the prepared datasets are provided in the next section; the interested student can explore the landing pages for the source datasets as follows:

1.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210035901">Canadian Principal Crops (Data & Metadata)</a>
2.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210007701">Farm product prices (Data & Metadata)</a>
3.  <a href="https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01">Bank of Canada daily average exchange rates</a>


### 1. Canadian Principal Crops Data *

This dataset contains agricultural production measures for the principle crops grown in Canada, including a breakdown by province and teritory, for each year from 1908 to 2020.

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A detailed description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901  
Detailed information is included in the metadata file and as header text in the data file, which can be downloaded - look for the 'download options' link.  

### 2. Farm product prices

This dataset contains monthly average farm product prices for Canadian crops and livestock by province and teritory, from 1980 to 2020 (or 'last year', whichever is greatest).

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210007701 
The information is included in the metadata file, which can be downloaded - look for the 'download options' link.  

### 3. Bank of Canada daily average exchange rates *

This dataset contains the daily average exchange rates for multiple foreign currencies. Exchange rates are expressed as 1 unit of the foreign currency converted into Canadian dollars. It includes only the latest four years of data, and the rates are published once each business day by 16:30 ET.

For this assignment you will use a snapshot of this dataset with only the USD-CAD exchange rates included (see next section). We have also prepared a monthly averaged version which you will be using below.

A brief description of this dataset and the original dataset can be obtained from the Bank of Canada Data Portal at:
https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates/

( * these datasets are the same as the ones you used in the practice lab)


### Dataset URLs

  1.  Annual Crop Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv 

  2.  Farm product prices: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv
  
  3.  Daily FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv
  
  4.  Monthly FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv
  

<span style="color:red">**IMPORTANT:**</span> You will be loading these datasets directly into R data frames from these URLs instead of from the StatsCan and Bank of Canada portals. The versions provided at these URLs are simplified and subsetted versions of the original datasets.


#### Now let's load these datasets into four separate Db2 tables.
Let's first load the RODBC package:


In [1]:
library(RODBC)

## Problem 1
#### Create tables
Establish a connection to the Db2 database, and create the following four tables using the RODBC package in R. 
Use the separate cells provided below to create each of your tables.

1.  **CROP_DATA**
2.  **FARM_PRICES**
3.  **DAILY_FX**
4.  **MONTHLY_FX**  

The previous practice lab will help you accomplish this.


### Solution 1


In [4]:
# Establish database connection
dsn_driver = "com.ibm.db2.jcc.DB2Driver"
dsn_database = "bludb"
dsn_hostname = "***"
dsn_port = "***"
dsn_protocol = "TCPIP"
dsn_uid = "****"
dsn_pwd = "*****"
dsn_security = "ssl"

conn_path = paste("DRIVER={IBM DB2 ODBC DRIVER}",
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")
conn = odbcDriverConnect(conn_path, believeNRows = FALSE)
conn

RODBC Connection 2
Details:
  case=nochange
  DRIVER={IBM DB2 ODBC DRIVER}
  UID=pwk99009
  PWD=******
  DATABASE=bludb
  HOSTNAME=21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud
  PORT=31864
  PROTOCOL=TCPIP
  SECURITY=SSL

In [7]:
tables <- c("CROP_DATA", "FARM_PRICES", "DAILY_FX", "MONTHLY_FX")

for (table in tables) {
        # Drop tables if they already exist
        out <- sqlTables(conn, tableType = "TABLE",
                          tableName = table)
        if (nrow(out)>0) {
            err <- sqlDrop(conn, table,
                            errors=FALSE)  
            if (err==-1) {
                cat("An error has occurred.\n")
                err.msg <- odbcGetErrMsg(conn)
                for (error in err.msg) { 
                    cat(error,"\n")
                }
            } 
            else {
                cat ("Table: ",table," was dropped\n")
            }
        }
        else {
              cat ("Table: ", table," does not exist\n")
        }
    }

Table:  CROP_DATA  was dropped
Table:  FARM_PRICES  was dropped
Table:  DAILY_FX  does not exist
Table:  MONTHLY_FX  does not exist


In [8]:
# CROP_DATA:
myquery <-"CREATE TABLE CROP_DATA (
            CD_ID INTEGER NOT NULL,
            YEAR DATE NOT NULL,
            CROP_TYPE VARCHAR(20) NOT NULL,
            GEO VARCHAR(20) NOT NULL, 
            SEEDED_AREA INTEGER NOT NULL,
            HARVESTED_AREA INTEGER NOT NULL,
            PRODUCTION INTEGER NOT NULL,
            AVG_YIELD INTEGER NOT NULL,
            PRIMARY KEY (CD_ID)
            )"

df1 <- sqlQuery(conn, myquery, errors=FALSE)

if (df1 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


In [9]:
# FARM_PRICES:
myquery <-"CREATE TABLE FARM_PRICES (
            CD_ID INTEGER NOT NULL,
            DATE DATE NOT NULL,
            CROP_TYPE VARCHAR(20) NOT NULL,
            GEO VARCHAR(20) NOT NULL,
            PRICE_PRERMT FLOAT(6) NOT NULL,
            PRIMARY KEY (CD_ID)
            )"

df2 <- sqlQuery(conn, myquery, errors=FALSE)

if (df2 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


In [10]:
# DAILY_FX:
df3 <- sqlQuery(conn, "CREATE TABLE DAILY_FX (
                                DFX_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL, 
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (DFX_ID)
                                )",
                    errors=FALSE
                    )

    if (df3 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


In [12]:
# MONTHLY_FX:
df4 <- sqlQuery(conn, "CREATE TABLE MONTHLY_FX (
                                DFX_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL, 
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (DFX_ID)
                                )",
                    errors=FALSE
                    )

    if (df4 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


## Problem 2
#### Read Datasets and Load Tables
Read the datasets into R dataframes using the urls provided above. Then load your tables.


###  Solution 2


In [13]:
crop_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv', colClasses=c(YEAR="character"))
farmprices_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv', colClasses=c(DATE="character"))
dailyfx_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv', colClasses=c(DATE="character"))
monthlyfx_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv', colClasses=c(DATE="character"))

In [14]:
tables <- c("crop_df", "farmprices_df", "dailyfx_df", "monthlyfx_df")
sql_tables <- c("CROP_DATA", "FARM_PRICES","DAILY_FX", "MONTHLY_FX")
for (i in 1:length(tables)) {
  table_name <- tables[i]
  sql_table_name <- sql_tables[i]
  
  # Execute the sqlSave command
  sqlSave(conn, get(table_name), sql_table_name, append = TRUE, fast = FALSE, rownames = FALSE, colnames = FALSE, verbose = FALSE)
}

In [3]:
tab.frame <- sqlTables(conn, schema="PWK99009")
nrow(tab.frame)
tab.frame$TABLE_NAME

[1] 10

[1] "BIKE_SHARING_SYSTEMS"    "BOARD"                  
 [3] "CITIES_WEATHER_FORECAST" "CROP_DATA"              
 [5] "DAILY_FX"                "FARM_PRICES"            
 [7] "MONTHLY_FX"              "SCHOOL"                 
 [9] "SEOUL_BIKE_SHARING"      "WORLD_CITIES"

## Now execute SQL queries using the RODBC R package to solve the assignment problems.

## Problem 3
#### How many records are in the farm prices dataset?


### Solution 3


In [17]:
sqlQuery(conn, "SELECT COUNT(CD_ID) AS RECORDS FROM FARM_PRICES;")

,RECORDS
,<int>
1,2678


## Problem 4
#### Which geographies are included in the farm prices dataset?


### Solution 4


In [23]:
sqlQuery(conn, "SELECT DISTINCT(GEO) AS DISTINCT_GEOS FROM FARM_PRICES;")

,DISTINCT_GEOS
,<fct>
1,Alberta
2,Saskatchewan


## Problem 5
#### How many hectares of Rye were harvested in Canada in 1968?


### Solution 5


In [26]:
sqlQuery(conn, "SELECT SUM(HARVESTED_AREA) AS TOTAL_HARVESTED_AREA_1968 FROM CROP_DATA WHERE YEAR(YEAR)=1968;")

,TOTAL_HARVESTED_AREA_1968
,<int>
1,29984800


## Problem 6
#### Query and display the first 6 rows of the farm prices table for Rye.


### Solution 6


In [27]:
sqlQuery(conn, "SELECT * FROM FARM_PRICES WHERE CROP_TYPE='Rye' LIMIT 6;")

,CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
,<int>,<date>,<fct>,<fct>,<dbl>
1,4,1985-01-01,Rye,Alberta,100.77
2,5,1985-01-01,Rye,Saskatchewan,109.75
3,10,1985-02-01,Rye,Alberta,95.05
4,11,1985-02-01,Rye,Saskatchewan,103.46
5,16,1985-03-01,Rye,Alberta,96.77
6,17,1985-03-01,Rye,Saskatchewan,106.38


## Problem 7
#### Which provinces grew Barley? 


### Solution 7


In [31]:
sqlQuery(conn, "SELECT DISTINCT(GEO) AS BARLEY_PROVINCES FROM FARM_PRICES WHERE CROP_TYPE='Barley';")

,BARLEY_PROVINCES
,<fct>
1,Alberta
2,Saskatchewan


## Problem 8
#### Find the first and last dates for the farm prices data.


### Solution 8


In [33]:
sqlQuery(conn, "SELECT MIN(DATE) AS MIN_DATE, MAX(DATE) AS MAX_DATE FROM FARM_PRICES;")

,MIN_DATE,MAX_DATE
,<date>,<date>
1,1985-01-01,2020-12-01


In [64]:
sqlQuery(conn, "SELECT MIN(YEAR) AS MIN_DATE, MAX(YEAR) AS MAX_DATE FROM CROP_DATA;")

,MIN_DATE,MAX_DATE
,<date>,<date>
1,1965-12-31,2020-12-31


## Problem 9
#### Which crops have ever reached a farm price greater than or equal to &#0036;350 per metric tonne?


### Solution 9


In [34]:
sqlQuery(conn, "SELECT DISTINCT(CROP_TYPE) AS CROPS_ATLEAST$350 FROM FARM_PRICES WHERE PRICE_PRERMT>=350;")

,CROPS_ATLEAST$350
,<fct>
1,Canola


## Problem 10
#### Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best? 


### Solution 10


In [51]:
sqlQuery(conn, "SELECT CROP_TYPE, 
                    AVG(AVG_YIELD) AS AVG_YIELD_2000 
                FROM CROP_DATA 
                WHERE YEAR(YEAR)=2000 
                GROUP BY CROP_TYPE
                ORDER BY AVG(AVG_YIELD) DESC;")

# Barley performed best

,CROP_TYPE,AVG_YIELD_2000
,<fct>,<int>
1,Barley,2933
2,Wheat,2366
3,Rye,2166
4,Canola,1466


## Problem 11
#### Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000? 


### Solution 11


In [52]:
sqlQuery(conn, "SELECT CROP_TYPE, GEO,
                    AVG(AVG_YIELD) AS AVG_YIELD_SINCE_2000
                FROM CROP_DATA 
                WHERE YEAR(YEAR)>=2000 
                GROUP BY CROP_TYPE, GEO
                ORDER BY AVG(AVG_YIELD) DESC;")

# Barley planted in alberta had the highest average yield

,CROP_TYPE,GEO,AVG_YIELD_SINCE_2000
,<fct>,<fct>,<int>
1,Barley,Alberta,3450
2,Barley,Canada,3253
3,Wheat,Alberta,3100
4,Barley,Saskatchewan,2971
5,Wheat,Canada,2845
6,Rye,Alberta,2683
7,Rye,Canada,2543
8,Wheat,Saskatchewan,2429
9,Rye,Saskatchewan,2226


## Problem 12
#### Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data.


### Solution 12


In [5]:
sqlQuery(conn, "SELECT CROP_TYPE, GEO,
                    SUM(PRODUCTION) AS SUM_PRODUCTION,
                    YEAR(YEAR) AS RECENT_YEAR
                FROM CROP_DATA
                WHERE GEO='Canada'
                    AND CROP_TYPE='Wheat'
                    AND YEAR(YEAR)=(SELECT MAX(YEAR(YEAR)) FROM CROP_DATA)
                GROUP BY CROP_TYPE, GEO, YEAR;")

# 3512

,CROP_TYPE,GEO,SUM_PRODUCTION,RECENT_YEAR
,<fct>,<fct>,<int>,<int>
1,Wheat,Canada,35183000,2020


## Problem 13
#### Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars. Display the most recent 6 months of the data.


### Solution 13


In [75]:
sqlQuery(conn, "SELECT CROP_TYPE, GEO, F.DATE,
                    F.PRICE_PRERMT AS PRICE_PRERMT_CAD,
                    F.PRICE_PRERMT/M.FXUSDCAD AS PRICE_PRERMT_USD
                FROM MONTHLY_FX M, FARM_PRICES F
                WHERE YEAR(M.DATE)=YEAR(F.DATE) AND MONTH(M.DATE)=MONTH(F.DATE)
                    AND GEO='Saskatchewan'
                    AND CROP_TYPE='Canola'
                    AND F.DATE >(SELECT MAX(DATE) - 6 MONTHS FROM FARM_PRICES)
                GROUP BY CROP_TYPE, GEO, F.DATE, F.PRICE_PRERMT, F.PRICE_PRERMT/M.FXUSDCAD
        ;")

,CROP_TYPE,GEO,DATE,PRICE_PRERMT_CAD,PRICE_PRERMT_USD
,<fct>,<fct>,<date>,<dbl>,<dbl>
1,Canola,Saskatchewan,2020-07-01,462.88,342.9122
2,Canola,Saskatchewan,2020-09-01,463.52,350.4056
3,Canola,Saskatchewan,2020-08-01,464.60,351.3827
4,Canola,Saskatchewan,2020-10-01,474.80,359.2966
5,Canola,Saskatchewan,2020-11-01,495.64,379.2718
6,Canola,Saskatchewan,2020-12-01,507.33,396.1130


## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                                          |
| ---------- | ------- | ------------- | ----------------------------------------------------------------------------------------------------------- |
| 2021-04-01 | 0.7     | Jeff Grossman | Split Problem 1 solution cell into multiple cells, fixed minor bugs |
| 2021-03-12 | 0.6     | Jeff Grossman | Cleaned up content for production |
| 2021-03-11 | 0.5     | Jeff Grossman | Moved more advanced problems to optional honours module |
| 2021-03-10 | 0.4     | Jeff Grossman | Added introductory and intermediate level problems and removed some advanced problems |
| 2021-03-04 | 0.3     | Jeff Grossman | Moved some problems to a new practice lab as prep for this assignment
| 2021-03-04 | 0.2     | Jeff Grossman | Sorted problems roughly by level of difficulty and relegated more advanced ones to ungraded bonus problems  |
| 2021-02-20 | 0.1     | Jeff Grossman | Started content creation                                                                                    |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>


In [76]:
close(conn)